**Data Cleaning**

---




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Télécharge les stopwords arabes à partir du corpus NLTK (Natural Language Toolkit).

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Importer les bibliothèques

In [ ]:
import re
import pandas as pd

Importe les fonctions pour les stopwords et la tokenization de NLTK.

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Installer Pyarabic - Fournit des fonctionnalités pour travailler avec du texte en arabe

In [ ]:
!pip install pyarabic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 1.3 MB/s eta 0:00:00


In [ ]:
from pyarabic.araby import *

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/HessPress Data Science Project/Data Visualization/cleaned_dataset2.csv')

Supprime les lignes en double et les valeurs manquantes dans le jeu de données.

In [ ]:
data = data.drop_duplicates().dropna()

Fonction prend du texte en entrée et utilise une expression régulière pour supprimer tous les caractères qui ne sont pas alphanumériques, espaces ou la virgule arabe (،).

In [1]:
def clean_text(text):

    return re.sub(r'[^\w\s،]', '', text)

In [ ]:
data['title'] = data['title'].apply(clean_text)
data['comment'] = data['comment'].apply(clean_text)
data['category'] = data['category'].apply(clean_text)

Affiche les valeurs uniques de la colonne "category".

In [ ]:
print(data['category'].unique())

En sauvegarde les données nettoyées dans un fichier 'cleaned_dataset2.csv'.

In [ ]:
data.to_csv('cleaned_dataset2.csv', index=False)

Defini une liste de ponctuations arabes et créent un modèle de regex pour les reconnaître dans le texte.

In [ ]:
punctuations = r"""!@#$%^&*()_-+={}[]|\:;'"<>,.?/~؟؛،«»""ـ"""
punctuations_pattern = "[" + re.escape(punctuations) + "]"
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

Fonction prend du texte en entrée et utilise la fonction re.sub() pour remplacer toutes les occurrences des caractères de ponctuation spécifiés dans punctuations_pattern par un espace dans le texte.

In [ ]:
def remove_punctuations(text):
    return re.sub(punctuations_pattern, ' ', text)

Fonction prend du texte en entrée et utilise la fonction re.sub() pour remplacer plusieurs variantes de lettres arabes (par exemple, "إ", "أ", "آ") par une forme standard. Cela vise à normaliser le texte en remplaçant les variantes de lettres par une seule forme standard.

In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

Fonction prend du texte en entrée et utilise la fonction re.sub() pour supprimer tous les diacritiques arabes (par exemple, Chada, Fatha, Damma, etc.) du texte.

In [ ]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

Fonction prend du texte en entrée et applique plusieurs étapes de prétraitement sur le texte. Elle utilise les fonctions normalize_arabic(), remove_punctuations(), et remove_diacritics() définies précédemment pour normaliser le texte, supprimer les ponctuations et supprimer les diacritiques, respectivement. Ensuite, elle utilise une expression régulière pour remplacer plusieurs espaces consécutifs par un seul espace, et enfin elle supprime les espaces de début et de fin du texte.

In [ ]:
def text_preprocessing(text):
    text = normalize_arabic(text)
    text = remove_punctuations(text)
    text = remove_diacritics(text)
    text = text.replace('\n', '')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
data['comment'] = data['comment'].apply(text_preprocessing)

In [ ]:
data.to_csv('new.csv', index=False)

In [ ]:
df = pd.read_csv('new.csv')

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
translation_table = str.maketrans(arabic_punctuations, ' ' * len(arabic_punctuations))
df['comment'] = df['comment'].str.translate(translation_table)
df['comment'] = df['comment'].str.replace(r'[a-zA-Z]', '')

In [ ]:
stop_words = set(stopwords.words('arabic'))

In [ ]:
def normalize_arabic_text(text):
    normalized_text = strip_tashkeel(text)
    normalized_text = normalize_ligature(normalized_text)
    normalized_text = strip_tatweel(normalized_text)
    return normalized_text

In [ ]:
def remove_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

In [ ]:
def normalize_letters(text):
  text = re.sub('[ى]', 'ي', text)
  text = re.sub('[إأٱآا]', 'ا', text)
  return text

In [ ]:
def remove_newlines_and_tabs(text):
    cleaned_text = text.replace('\n', '').replace('\t', '')
    return cleaned_text

In [ ]:
df['comment'] = df['comment'].apply(normalize_arabic_text)
df['comment'] = df['comment'].apply(remove_stopwords)
df['comment'] = df['comment'].apply(normalize_letters)

In [ ]:
# Ajouter des colonnes pour compter les mots et les lettres
df['word_count'] = df['comment'].apply(lambda x: len(x.split()))
df['letter_count'] = df['comment'].apply(lambda x: len(x.replace(' ', '')))

In [ ]:
# Sauvegarder les données finales
df.to_csv('finalprocessing2.csv', index=False)

In [ ]:
# Afficher les premières lignes du DataFrame final
new_df = pd.read_csv('finalprocessing2.csv')
new_df.head()